In [2]:
#@title Get text train and test ,X and Y
nrows_train = 20000 # @param {type:"integer"}
sequence_length = 13 # @param {type:"integer"}

feed_lenght = 8

import pandas as pd
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import json
from keras.preprocessing.text import tokenizer_from_json

#load fusion
fus = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/airport_ml/LECO/notebooks/LECOfusionml.csv",
                  parse_dates=["time"], index_col="time")

#Get the train and test train
texts_train = fus["fusion"].sample(nrows_train,)
texts_test = fus["fusion"].drop(texts_train.index)

#save texts test
texts_test.to_csv("/content/drive/MyDrive/Colab Notebooks/airport_ml/LECO/notebooks/LECOtexts_testml.csv")

# Tokenize text
tokenizer = Tokenizer()

#tokenizer.fit_on_texts(texts_train)
tokenizer.fit_on_texts(fus["fusion"])

#Save tokenizer
tokenizer_json = tokenizer.to_json()

# Save the JSON configuration to a file
with open('/content/drive/MyDrive/Colab Notebooks/airport_ml/LECO/algorithms/LECOtokenizerml.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

sequences = tokenizer.texts_to_sequences(texts_train)

# Prepare input and output data
X = []
y = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        x_seq = sequence[:i]
        x_seq_padded = pad_sequences([x_seq], maxlen=sequence_length, padding='pre')
        X.append(x_seq_padded[0])
        y.append(sequence[i])

X = np.array(X)
y = np.array(y)

#filter seed/model words
df = pd.DataFrame(X)
df["y"] = y
df_fil =  df[(df.iloc[:, :sequence_length-feed_lenght+1] != 0).any(axis=1)]
X = df_fil.iloc[:, :-1].values
y = df_fil.iloc[:, -1].values

# One hot encode the outputs
y = np.eye(len(tokenizer.word_index) + 1)[y]

print("Dictionary size (words): len(y[i]):",len(y[1]) )
print("Total X variables:",len(X) )

print("\nText train example1")
display(texts_train[0])

print("\nSequences example 1")
display(sequences[0])

print("\nText train example2")
display(texts_train[1])

print("\nSequences example 2")
display(sequences[1])

print("\nX")
display(X[:25])

print("\nDictionary first 10 words")
display({k: tokenizer.word_index[k] for k in list(tokenizer.word_index)[:10]})


Dictionary size (words): len(y[i]): 3358
Total X variables: 163552

Text train example1


'26003kt 9999 WM NSCNClD M 13 10 q1023 19004KT 9000 NSC 13/11 Q1023 NOSIG'


Sequences example 1


[436, 1, 2, 76, 3, 6, 12, 38, 210, 83, 74, 6, 10, 38, 4]


Text train example2


...

[270, 1, 2, 21, 31, 8, 10, 43, 199, 1129, 1, 21, 7, 15, 40, 4]


X


array([[   0,    0,    0,    0,    0,  436,    1,    2,   76,    3,    6,
          12,   38],
       [   0,    0,    0,    0,  436,    1,    2,   76,    3,    6,   12,
          38,  210],
       [   0,    0,    0,  436,    1,    2,   76,    3,    6,   12,   38,
         210,   83],
       [   0,    0,  436,    1,    2,   76,    3,    6,   12,   38,  210,
          83,   74],
       [   0,  436,    1,    2,   76,    3,    6,   12,   38,  210,   83,
          74,    6],
       [ 436,    1,    2,   76,    3,    6,   12,   38,  210,   83,   74,
           6,   10],
       [   1,    2,   76,    3,    6,   12,   38,  210,   83,   74,    6,
          10,   38],
       [   0,    0,    0,    0,    0,  270,    1,    2,   21,   31,    8,
          10,   43],
       [   0,    0,    0,    0,  270,    1,    2,   21,   31,    8,   10,
          43,  199],
       [   0,    0,    0,  270,    1,    2,   21,   31,    8,   10,   43,
         199, 1129],
       [   0,    0,  270,    1,    2,   21,   31, 


Dictionary first 10 words


{'9999': 1,
 'wm': 2,
 'm': 3,
 'nosig': 4,
 '12': 5,
 '13': 6,
 '16': 7,
 '15': 8,
 '14': 9,
 '11': 10}

In [3]:
#@title Train the model

from keras.layers import Bidirectional

# Build the LSTM model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=20, input_length=sequence_length),
    Bidirectional(LSTM(130)),

    Dense(len(tokenizer.word_index)+1, activation='softmax')
])


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=70,batch_size=512)

#save the model
model.save("/content/drive/MyDrive/Colab Notebooks/airport_ml/LECO/algorithms/LECOml.keras")


Epoch 1/70
320/320 [==============================] - 24s 65ms/step - loss: 3.8104 - accuracy: 0.2305
Epoch 2/70
320/320 [==============================] - 21s 64ms/step - loss: 2.7907 - accuracy: 0.3147
Epoch 3/70
320/320 [==============================] - 21s 65ms/step - loss: 2.4714 - accuracy: 0.3578
Epoch 4/70
320/320 [==============================] - 21s 66ms/step - loss: 2.2534 - accuracy: 0.3975
Epoch 5/70
320/320 [==============================] - 21s 64ms/step - loss: 2.1380 - accuracy: 0.4167
Epoch 6/70
320/320 [==============================] - 21s 65ms/step - loss: 2.0450 - accuracy: 0.4331
Epoch 7/70
320/320 [==============================] - 21s 64ms/step - loss: 1.9635 - accuracy: 0.4477
Epoch 8/70
320/320 [==============================] - 21s 66ms/step - loss: 1.8924 - accuracy: 0.4619
Epoch 9/70
320/320 [==============================] - 21s 65ms/step - loss: 1.8264 - accuracy: 0.4748
Epoch 10/70
320/320 [==============================] - 21s 65ms/step - loss: 1.765